# TECFIDERA PATIENT CONTINUOUS ENROLLMENT
This notebook pulls all the tecfidera drug patients from each source and checks the continuous enrollment for the patient.
If patient is coming for refill before tecfidera dosage enddate then those patients will be assumed as continuous.
Also, we have taken buffer of 60 days i.e if the patient is coming for refill before 60 days of drug completion/enddate then it will taken as continuous.

## Set up Notebook
The following step is essential to set up the notebook to do everything that you need. In the following code snow = Snowflake(role = 'RWD_SANDBOX',database='RWD_DB', schema='SANDBOX') the connections are set to hook up to the correct Role, database, and schema in snowfalke. Please note, these are just the default values and in future queries you can change the database and schema if the role hass access to them.

In [1]:
import qgrid
from datetime import timedelta, datetime
import math
import pandas as pd
from drg_connect import Snowflake

snow = Snowflake(role = 'PROJECT_BIOGEN_CUSTOM_ROLE',database='PROJECT_ANALYTICS',schema = 'BIOGEN')
engine = snow.engine

%load_ext sql_magic
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine

## Initializing standard variables
These will be used throughout the project

In [2]:
buffer_60 = 60.0 
buffer_90 = 90.0 
buffer_120 = 120.0 
to_sql_array = lambda x: "({})".format(str(x)[1:-1])  # handy little function!
ndc_codes = to_sql_array(['64406000703','64406000501','64406000602'])
db = "PROJECT_ANALYTICS"
schema = "BIOGEN"
char_tbl = "Akmishra_TECFIDERA"
sith_ref_tbl = "PROJECT_BIOGEN_CUSTOM_ROLE"

## Initial Data Pull 
Extrat data from all sources for the patients taking tecfidera drug

### Pharmacy Data Pull (Condor+Osprey)

In [8]:
%%read_sql
--Pulling all the patients taking techfidera drugs from Condor+Osprey pharmacy table.
CREATE TABLE {db}.{schema}.{char_tbl}_PHARMACY_V1  AS
      SELECT 'PHARMACY'                              AS SOURCE
             ,CLAIM_NUMBER
             ,ENCRYPTED_KEY_1
             ,ENCRYPTED_KEY_2
             ,DATE_OF_BIRTH                         AS PATIENT_DOB
             ,CASE WHEN PATIENT_GENDER_CODE  IN('1','M')
                   THEN 'M'
                   WHEN PATIENT_GENDER_CODE  IN('2','F')
                   THEN 'F'
                   ELSE 'X'
              END                                   AS PATIENT_GENDER
             ,PRODUCT_OR_SERVICE_ID                 AS DRUG_CODE
             ,DATE_OF_SERVICE                       AS SERVICE_DATE
             ,DAYS_SUPPLY

       FROM  RWD_DB.RWD.RAVEN_PHARMACY
       WHERE PRODUCT_OR_SERVICE_ID           IN {ndc_codes}
         AND UPPER(RESPONSE_CODE)            = 'P' 
;

Query started at 11:20:03 PM India Standard Time; Query executed in 32.96 m

,status
0,Table AKMISHRA_PHARMACY_TECHFIDERA_V1 successf...


Count of patients and unique pharmacy claims from condor and osprey.  We have roughly 50k patients with over 1 million claims.

In [85]:
%%read_sql
--count of distinct patients and distinct claims from pharmacy data
    SELECT COUNT(DISTINCT ENCRYPTED_KEY_1) AS PATIENT_COUNT,
           COUNT(DISTINCT CLAIM_NUMBER)    AS CLAIM_COUNT
      FROM {db}.{schema}.{char_tbl}_PHARMACY_V1
     WHERE ENCRYPTED_KEY_1 <> ' '
       AND ENCRYPTED_KEY_1 IS NOT NULL
       AND ENCRYPTED_KEY_1 <> 'NULL'
          

Query started at 11:53:05 PM India Standard Time; Query executed in 0.07 m

,patient_count,claim_count
0,48344,1034754


There are three ndc codes for Tecfidera.  One is the starter pack and we need to figure out which is which

In [86]:
%%read_sql
--count of distinct patients and distinct claims for each ndc from pharmacy data 
    SELECT DRUG_CODE ,
           COUNT(DISTINCT ENCRYPTED_KEY_1) AS PATIENT_COUNT,
           COUNT(DISTINCT CLAIM_NUMBER)    AS CLAIM_COUNT
      FROM {db}.{schema}.{char_tbl}_PHARMACY_V1
     WHERE ENCRYPTED_KEY_1 <> ' '
       AND ENCRYPTED_KEY_1 IS NOT NULL
       AND ENCRYPTED_KEY_1 <> 'NULL'
    GROUP BY DRUG_CODE
;   

Query started at 11:53:16 PM India Standard Time; Query executed in 0.20 m

,drug_code,patient_count,claim_count
0,64406000602,44404,952248
1,64406000501,3554,16747
2,64406000703,26098,65759


### Medical Data Pull (Raven + Osprey)
Extract the ndc codes for Tecfidera from the raven procedures.  This is likely going to be a small number

In [83]:
%%read_sql
--Pulling all the patients taking techfidera drugs from RAVEN table.
CREATE TABLE {db}.{schema}.{char_tbl}_MEDICAL_V1 AS
      SELECT 'Medical'                                                                            AS SOURCE
             ,CLAIM_NUMBER
             ,ENCRYPTED_KEY_1
             ,ENCRYPTED_KEY_2
             ,NDC                                                                                 AS DRUG_CODE
             ,COALESCE(STATEMENT_FROM,STATEMENT_TO,RECEIVED_DATE)                                 AS SERVICE_DATE
             ,'0.0'                                                                               AS DAYS_SUPPLY

       FROM  RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_PROCEDURE
       WHERE NDC IN {ndc_codes}
;

Query started at 08:57:06 PM India Standard Time

Exception during reset or similar
Traceback (most recent call last):
  File "C:\Users\akmishra\AppData\Local\Continuum\anaconda3\lib\site-packages\sqlalchemy\pool.py", line 703, in _finalize_fairy
    fairy._reset(pool)
  File "C:\Users\akmishra\AppData\Local\Continuum\anaconda3\lib\site-packages\sqlalchemy\pool.py", line 873, in _reset
    pool._dialect.do_rollback(self)
  File "C:\Users\akmishra\AppData\Local\Continuum\anaconda3\lib\site-packages\sqlalchemy\engine\default.py", line 457, in do_rollback
    dbapi_connection.rollback()
  File "C:\Users\akmishra\AppData\Local\Continuum\anaconda3\lib\site-packages\snowflake\connector\connection.py", line 384, in rollback
    self.cursor().execute("ROLLBACK")
  File "C:\Users\akmishra\AppData\Local\Continuum\anaconda3\lib\site-packages\snowflake\connector\cursor.py", line 479, in execute
    _is_put_get=_is_put_get)
  File "C:\Users\akmishra\AppData\Local\Continuum\anaconda3\lib\site-packages\snowflake\connector\cursor.py", line 368, in _e

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


Exception during reset or similar
Traceback (most recent call last):
  File "C:\Users\akmishra\AppData\Local\Continuum\anaconda3\lib\site-packages\sqlalchemy\engine\base.py", line 1193, in _execute_context
    context)
  File "C:\Users\akmishra\AppData\Local\Continuum\anaconda3\lib\site-packages\sqlalchemy\engine\default.py", line 507, in do_execute
    cursor.execute(statement, parameters)
  File "C:\Users\akmishra\AppData\Local\Continuum\anaconda3\lib\site-packages\snowflake\connector\cursor.py", line 542, in execute
    errvalue)
  File "C:\Users\akmishra\AppData\Local\Continuum\anaconda3\lib\site-packages\snowflake\connector\errors.py", line 97, in errorhandler_wrapper
    cursor.errorhandler(connection, cursor, errorclass, errorvalue)
  File "C:\Users\akmishra\AppData\Local\Continuum\anaconda3\lib\site-packages\snowflake\connector\errors.py", line 73, in default_errorhandler
    done_format_msg=errorvalue.get(u'done_format_msg'))
snowflake.connector.errors.ProgrammingError: 390420

ProgrammingError: (snowflake.connector.errors.ProgrammingError) 390420: IP 103.248.174.119 is not allowed to access Snowflake.  Contact your local security administrator. (Background on this error at: http://sqlalche.me/e/f405)

This is such a small number that it is not worth brining in. Tecfidera is a drug that is typicallly ordered from the pharmacy and it would rarely show up in the procedures table. We'll keep it but it likley wasn't necessary to bring in

In [87]:
%%read_sql
--count of distinct patients and distinct claims from MEDICAL data
    SELECT COUNT(DISTINCT ENCRYPTED_KEY_1) AS PATIENT_COUNT,
           COUNT(DISTINCT CLAIM_NUMBER)    AS CLAIM_COUNT
      FROM {db}.{schema}.{char_tbl}_MEDICAL_V1
     WHERE ENCRYPTED_KEY_1 <> ' '
           AND ENCRYPTED_KEY_1 IS NOT NULL
           AND ENCRYPTED_KEY_1 <> 'NULL'
          

Query started at 11:53:39 PM India Standard Time; Query executed in 0.17 m

,patient_count,claim_count
0,119,1453


These are extremely small counts and next time we will skip brining it in.

In [88]:
%%read_sql
--count of distinct patients and distinct claims for each ndc from MEDICAL data 
    SELECT DRUG_CODE ,
           COUNT(DISTINCT ENCRYPTED_KEY_1) AS PATIENT_COUNT,
           COUNT(DISTINCT CLAIM_NUMBER)    AS CLAIM_COUNT
      FROM PROJECT_ANALYTICS.Biogen.AKMISHRA_MEDICAL_TECHFIDERA_V1
     WHERE ENCRYPTED_KEY_1 <> ' '
       AND ENCRYPTED_KEY_1 IS NOT NULL
       AND ENCRYPTED_KEY_1 <> 'NULL'
    GROUP BY DRUG_CODE
;   

Query started at 11:53:53 PM India Standard Time; Query executed in 0.16 m

,drug_code,patient_count,claim_count
0,64406000602,111,1395
1,64406000501,4,6
2,64406000703,34,51


### Pelican EHR Data Pull
All tecfidera records will be pulled from the EHR data.  

In [9]:
%%read_sql
--Pulling all the patients taking techfidera drugs from Pelican table.
CREATE Or Replace TABLE {db}.{schema}.{char_tbl}_PELICAN_V1 AS
      SELECT 'Pelican_EHR'       AS SOURCE
             ,B.PATIENT_ID
             ,A.MEDICATION_ID    AS DRUG_CODE
             ,B.START_DATE       AS SERVICE_DATE
             ,'0.0'              AS DAYS_SUPPLY
             ,PRESCRIPTION_ID

       FROM  rwd_db.rwd.PELICAN_MEDICATION A
             LEFT JOIN rwd_db.rwd.PELICAN_PRESCRIPTION B 
                    ON A.MEDICATION_ID   = B.MEDICATION_ID
       WHERE A.MEDICATION_ID IN {ndc_codes}
;

Query started at 05:30:49 PM India Standard Time; Query executed in 0.36 m

,status
0,Table AKMISHRA_PELICAN_TECHFIDERA_V1 successfu...


There are only 11,000 rows which is kind of small but we'll bring it in. 

In [10]:
%%read_sql

--Pulling Patient_ID(ENCRYPTED_KEY_1,ENCRYPTED_KEY_2) for each Patient_ID

    ALTER TABLE {db}.{schema}.{char_tbl}_PELICAN_V1
      ADD (ENCRYPTED_KEY_1 VARCHAR(),
           ENCRYPTED_KEY_2 VARCHAR()
          );

     UPDATE {db}.{schema}.{char_tbl}_PELICAN_V1                     A
        SET A.ENCRYPTED_KEY_1 = B.ENCRYPTED_KEY_1,
            A.ENCRYPTED_KEY_2 = B.ENCRYPTED_KEY_2
       FROM rwd_db.rwd.PELICAN_DEID                                  B
      WHERE A.PATIENT_ID = B.PATIENT_ID
      ;
       

Query started at 05:31:25 PM India Standard Time; Query executed in 0.04 mQuery started at 05:31:27 PM India Standard Time; Query executed in 0.98 m

,number of rows updated,number of multi-joined rows updated
0,11168,0


It appears that some of the pelican tecfidera orders/scripts have a claim number.  I'm not sure what that means.

In [19]:
%%read_sql

--Pulling Claim_Number(TRANSCRIPT_ID) for each Prescription

    ALTER TABLE {db}.{schema}.{char_tbl}_PELICAN_V1
      ADD (CLAIM_NUMBER VARCHAR()
          );

     UPDATE {db}.{schema}.{char_tbl}_PELICAN_V1                             A
        SET A.CLAIM_NUMBER = B.TRANSCRIPT_ID
       FROM rwd_db.rwd.PELICAN_TRANSCRIPT_PRESCRIPTION                       B
      WHERE A.PRESCRIPTION_ID = B.PRESCRIPTION_ID
      ;

Query started at 05:51:27 PM India Standard Time; Query executed in 0.04 mQuery started at 05:51:29 PM India Standard Time; Query executed in 0.38 m

,number of rows updated,number of multi-joined rows updated
0,3190,1406


I'm not sure why there are fewer claims than patients.  I'm assuming that most of the Tecfidera didn't match to a claim.
Per Akansha Pelican tends to be a bit smaller

In [90]:
%%read_sql
--count of distinct patients and distinct claims from PELICAN data
    SELECT COUNT(DISTINCT ENCRYPTED_KEY_1) AS PATIENT_COUNT,
           COUNT(DISTINCT CLAIM_NUMBER)    AS CLAIM_COUNT
      FROM {db}.{schema}.{char_tbl}_PELICAN_V1
     WHERE ENCRYPTED_KEY_1 <> ' '
       AND ENCRYPTED_KEY_1 IS NOT NULL
       AND ENCRYPTED_KEY_1 <> 'NULL'
          

Query started at 12:20:46 AM India Standard Time; Query executed in 0.12 m

,patient_count,claim_count
0,7356,3116


In [91]:
%%read_sql
--count of distinct patients and distinct claims for each ndc from PELICAN data 
    SELECT DRUG_CODE ,
           COUNT(DISTINCT ENCRYPTED_KEY_1) AS PATIENT_COUNT,
           COUNT(DISTINCT CLAIM_NUMBER)    AS CLAIM_COUNT
      FROM {db}.{schema}.{char_tbl}_PELICAN_V1
     WHERE ENCRYPTED_KEY_1 <> ' '
       AND ENCRYPTED_KEY_1 IS NOT NULL
       AND ENCRYPTED_KEY_1 <> 'NULL'
    GROUP BY DRUG_CODE
;   

Query started at 12:20:58 AM India Standard Time; Query executed in 0.10 m

,drug_code,patient_count,claim_count
0,64406000703,556,228
1,64406000501,1216,416
2,64406000602,6101,2518


### Albatross EHR Data Pull
The tecfidera orders from the EHR were extracted.  It appears we're pulling too many statuses per what Cindi Gettman recommended; however, our logic later to determine continuous enrollment actually may make this not a big deal. This may need to be revisited but for now it can remain.

In [12]:
%%read_sql
--Pulling all the patients taking techfidera drugs from Albatross table.
CREATE TABLE {db}.{schema}.{char_tbl}_ALBATROSS_V2 AS
      SELECT 'Albatross_EHR'                                                                      AS SOURCE
             ,CAST(ENCOUNTERID AS STRING)                                                         AS CLAIM_NUMBER
             ,ENCRYPTED_KEY_1
             ,ENCRYPTED_KEY_2
             ,NDC                                                                                 AS DRUG_CODE
             ,TO_DATE(RECORDEDDTTM)                                                               AS SERVICE_DATE
             ,daysupply                                                                           AS DAYS_SUPPLY
             ,refills                                                                             AS REFILLS
             ,quantitytodispense                                                                  AS QUANTITY_DISPENSED

       FROM  RWD_DB.RWD.ALBATROSS_EHR_MEDICATIONS                                                 A
                
       WHERE A.NDC             IN {ndc_codes}
         AND UPPER(STATUS)     IN ('','ACTIVE','ADMINISTERED','AUTO COMPLETE','COMPLETE','COMPLETED','CONTINUED',
                                   'DISCONTINUED','COMPLETE','COMPLETED','CONTINUED','DISCONTINUED','FILLED RX',
                                   'HOLD FOR','IN PROGRESS','NEED INFORMATION','ORDERED','OTHER','PENDING','RESULTED')
;

Query started at 03:12:36 PM India Standard TimeInitiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
; Query executed in 35.78 m

,status
0,Table AKMISHRA_TECFIDERA_ALBATROSS_V2 successf...


Not surpirsing we have 16k patients and 60k claims.  This is a high number and hopefully will overlap well with the lymphocyte tests.

In [13]:
%%read_sql
--count of distinct patients and distinct claims from MEDICAL data
    SELECT COUNT(DISTINCT ENCRYPTED_KEY_1) AS PATIENT_COUNT,
           COUNT(DISTINCT CLAIM_NUMBER)    AS CLAIM_COUNT
      FROM {db}.{schema}.{char_tbl}_ALBATROSS_V2
     WHERE ENCRYPTED_KEY_1 <> ' '
       AND ENCRYPTED_KEY_1 IS NOT NULL
       AND ENCRYPTED_KEY_1 <> 'NULL'
          

Query started at 04:11:04 PM India Standard Time; Query executed in 0.23 m

,patient_count,claim_count
0,16495,61341


The split between the drug codes is good.  We need to figure out which one is the starter pack.

In [14]:
%%read_sql
--count of distinct patients and distinct claims for each ndc from MEDICAL data 
    SELECT DRUG_CODE ,
           COUNT(DISTINCT ENCRYPTED_KEY_1) AS PATIENT_COUNT,
           COUNT(DISTINCT CLAIM_NUMBER)    AS CLAIM_COUNT
      FROM {db}.{schema}.{char_tbl}_ALBATROSS_V2
     WHERE ENCRYPTED_KEY_1 <> ' '
       AND ENCRYPTED_KEY_1 IS NOT NULL
       AND ENCRYPTED_KEY_1 <> 'NULL'
    GROUP BY DRUG_CODE
; 

Query started at 04:11:43 PM India Standard Time; Query executed in 0.16 m

,drug_code,patient_count,claim_count
0,64406000602,13822,48480
1,64406000501,2417,5839
2,64406000703,4980,11487


## Create final raw data

### Union raw Tecfidera Data
Combining tables from all the datasources into one master tables
* PRC - I never like performing massive unions of multiple data sources, and prefer using insert statements especially when data is large. However, for this size data it's an appropriate method.

In [16]:
%%read_sql

Create table {db}.{schema}.{char_tbl}_All_V2 as
    Select Source,
           CLAIM_NUMBER,
           ENCRYPTED_KEY_1,
           CASE WHEN ENCRYPTED_KEY_2 IS NULL THEN ENCRYPTED_KEY_1 
                ELSE CONCAT(ENCRYPTED_KEY_1,ENCRYPTED_KEY_2) END                        AS  PATIENT_SUFFIX,
            DRUG_CODE,
            SERVICE_DATE,
            Days_Supply,
            0       AS REFILLS,
            0       AS QUANTITY_DISPENSED
            
       FROM {db}.{schema}.{char_tbl}_Pharmacy_V1
      WHERE PATIENT_SUFFIX <> ' '
        AND PATIENT_SUFFIX IS NOT NULL
        AND PATIENT_SUFFIX <> 'NULL'
UNION ALL
    Select Source,
           CLAIM_NUMBER,
           ENCRYPTED_KEY_1,
           CASE WHEN ENCRYPTED_KEY_2 IS NULL THEN ENCRYPTED_KEY_1 
                ELSE CONCAT(ENCRYPTED_KEY_1,ENCRYPTED_KEY_2) END                        AS PATIENT_SUFFIX,
           
            DRUG_CODE,
            SERVICE_DATE,
            Days_Supply,
            0       AS REFILLS,
            0       AS QUANTITY_DISPENSED
            
       FROM {db}.{schema}.{char_tbl}_Medical_V1
      WHERE PATIENT_SUFFIX <> ' '
        AND PATIENT_SUFFIX IS NOT NULL
        AND PATIENT_SUFFIX <> 'NULL'

UNION ALL
    Select Source,
           CLAIM_NUMBER,
           ENCRYPTED_KEY_1,
           CASE WHEN ENCRYPTED_KEY_2 IS NULL THEN ENCRYPTED_KEY_1 
                ELSE CONCAT(ENCRYPTED_KEY_1,ENCRYPTED_KEY_2) END                        AS PATIENT_SUFFIX,
           
            DRUG_CODE,
            SERVICE_DATE,
            Days_Supply,
            0       AS REFILLS,
            0       AS QUANTITY_DISPENSED
            
       FROM {db}.{schema}.{char_tbl}_Pelican_V1
      WHERE PATIENT_SUFFIX <> ' '
        AND PATIENT_SUFFIX IS NOT NULL
        AND PATIENT_SUFFIX <> 'NULL'

UNION ALL
    Select Source,
           CLAIM_NUMBER,
           ENCRYPTED_KEY_1,
           CASE WHEN ENCRYPTED_KEY_2 IS NULL THEN ENCRYPTED_KEY_1 
                ELSE CONCAT(ENCRYPTED_KEY_1,ENCRYPTED_KEY_2) END                        AS PATIENT_SUFFIX,
           
            DRUG_CODE,
            SERVICE_DATE,
            Days_Supply,
            REFILLS,
            QUANTITY_DISPENSED
            
       FROM {db}.{schema}.{char_tbl}_ALBATROSS_V2
      WHERE PATIENT_SUFFIX <> ' '
        AND PATIENT_SUFFIX IS NOT NULL
        AND PATIENT_SUFFIX <> 'NULL'            

Query started at 04:16:05 PM India Standard Time; Query executed in 0.64 m

,status
0,Table AKMISHRA_TECFIDERA_ALL_V2 successfully c...


64k patients and 1.1 million claims is quite large. I'm not sure how this compares to our previous pull but for the most part it looks good

In [17]:
%%read_sql
--count of distinct patients and distinct claims from MEDICAL data
    SELECT COUNT(DISTINCT PATIENT_SUFFIX) AS PATIENT_COUNT,
           COUNT(DISTINCT CLAIM_NUMBER)    AS CLAIM_COUNT
      FROM {db}.{schema}.{char_tbl}_All_V2
       

Query started at 04:18:06 PM India Standard Time; Query executed in 0.08 m

,patient_count,claim_count
0,64221,1100664


The divide between the drug codes is useful information, and we may capitlize on it later.  

In [19]:
%%read_sql
--count of distinct patients and distinct claims from All data
    SELECT Drug_Code,
           COUNT(DISTINCT PATIENT_SUFFIX) AS PATIENT_COUNT,
           COUNT(DISTINCT CLAIM_NUMBER)    AS CLAIM_COUNT
      FROM {db}.{schema}.{char_tbl}_All_V2
      Group By 1
      ;
  

Query started at 04:18:37 PM India Standard Time; Query executed in 0.05 m

,drug_code,patient_count,claim_count
0,64406000602,57709,1004642
1,64406000501,6946,23008
2,64406000703,30678,77525


### Limit Data to ALC Patients
Reduce the Tecfidera population to only those patients who have at least 1 ALC test in the EHR.  Patients without any ALC tests would be excluded anyway and we want to eliminate patients to save computational time.

In [4]:
%%read_sql

DROP TABLE IF EXISTS {db}.{schema}.{char_tbl}_AlC_Final2;

CREATE TABLE {db}.{schema}.{char_tbl}_AlC_Final2 As
    SELECT A.* 
      FROM {db}.{schema}.{char_tbl}_ALL_V2      A
           INNER JOIN PROJECT_ANALYTICS.Biogen.cbg_ALC_Combined     B
                   ON A.PATIENT_SUFFIX  =  B.Patient_ID

Query started at 10:47:52 AM Eastern Daylight Time; Query executed in 0.05 mQuery started at 10:47:55 AM Eastern Daylight Time; Query executed in 0.20 m

,status
0,Table AKMISHRA_TECFIDERA_ALC_FINAL2 successful...


Reducing to 2700 patients with around 60k tecfidera orders is in line with what was done previously. This patient count is expected to drop further once we reduce to those that were on it continuously for 90 days and had a baseline ALC.  We may have a much smaller cohort than what we had previously.

In [5]:
%%read_sql
--count of distinct patients and distinct claims from Tec + ALC data 
    SELECT COUNT(DISTINCT PATIENT_SUFFIX) AS PATIENT_COUNT,
           COUNT(DISTINCT CLAIM_NUMBER)    AS CLAIM_COUNT
      FROM {db}.{schema}.{char_tbl}_AlC_Final2
       

Query started at 10:48:11 AM Eastern Daylight Time; Query executed in 0.19 m

,patient_count,claim_count
0,5318,68080


Counts of the drug codes are in line with what was done previously.

In [6]:
%%read_sql
--count of distinct patients and distinct claims from Tec + ALC  data
    SELECT Drug_code,
           COUNT(DISTINCT PATIENT_SUFFIX) AS PATIENT_COUNT,
           COUNT(DISTINCT CLAIM_NUMBER)    AS CLAIM_COUNT
      FROM {db}.{schema}.{char_tbl}_AlC_Final2 
     group by 1

Query started at 10:48:28 AM Eastern Daylight Time; Query executed in 0.15 m

,drug_code,patient_count,claim_count
0,64406000602,4679,59096
1,64406000501,942,2948
2,64406000703,2721,8016


## Continuous RX Analysis
Determine which med orders are continuous in this population

In [7]:
%%read_sql -d Akmishra_TECFIDERA

    SELECT Distinct SOURCE
          ,CLAIM_NUMBER
          ,PATIENT_SUFFIX
          ,SERVICE_DATE
          ,DRUG_CODE
          ,DAYS_SUPPLY
          ,REFILLS
          ,QUANTITY_DISPENSED
    FROM  {db}.{schema}.{char_tbl}_AlC_Final2
    WHERE PATIENT_SUFFIX IS NOT NULL
          AND SOURCE <> 'Pelican_EHR'
          AND source <> 'Medical'
          AND PATIENT_SUFFIX <> ''
          AND PATIENT_SUFFIX <> 'NULL'
          AND service_date > '3/1/2013'
          AND service_date is not null
    ORDER BY PATIENT_SUFFIX,SERVICE_DATE
;

Query started at 10:48:43 AM Eastern Daylight Time; Query executed in 0.45 m

In [8]:
#Delete where the service date is null
Akmishra_TECFIDERA = Akmishra_TECFIDERA.dropna(subset=['service_date'])

Set null days_supply and refills to 0 if the data is null

In [9]:
#Set the service_date to a date insetad of an object
Akmishra_TECFIDERA.service_date = pd.to_datetime(Akmishra_TECFIDERA.service_date)

In [10]:
#Set nulls to zero and then turn into numerics
Akmishra_TECFIDERA.loc[pd.isnull(Akmishra_TECFIDERA.days_supply),'days_supply'] = 0
Akmishra_TECFIDERA.loc[pd.isnull(Akmishra_TECFIDERA.refills),'refills'] = 0
Akmishra_TECFIDERA.loc[pd.isnull(Akmishra_TECFIDERA.quantity_dispensed),'quantity_dispensed'] = 0

Akmishra_TECFIDERA['days_supply'] = pd.to_numeric(Akmishra_TECFIDERA['days_supply'])
Akmishra_TECFIDERA['refills'] = pd.to_numeric(Akmishra_TECFIDERA['refills'])
Akmishra_TECFIDERA['quantity_dispensed'] = pd.to_numeric(Akmishra_TECFIDERA['quantity_dispensed'])


In [11]:
#Set days_supply_final to the max of the following
Akmishra_TECFIDERA['days_supply_final'] = (
    Akmishra_TECFIDERA.apply(lambda row: max(row.days_supply,
                                             row.quantity_dispensed / 2,
                                             row.refills * 30),
                             axis=1))

In [12]:
#Drop the records where the days_supply_final is zero.  These are likely bad records
Akmishra_TECFIDERA = Akmishra_TECFIDERA[Akmishra_TECFIDERA.days_supply_final > 0]

In [13]:
#Set end date (this was in a loop previously and is unnecesary)
Akmishra_TECFIDERA['end_date'] =  (
    Akmishra_TECFIDERA.apply(
        lambda row: row.service_date + timedelta(days=row.days_supply_final),
        axis=1))

In [14]:
%%read_sql
Drop table {db}.{schema}.Akmishra_TECFIDERA_Enddate

Query started at 10:50:15 AM Eastern Daylight Time; Query executed in 0.11 m

,status
0,AKMISHRA_TECFIDERA_ENDDATE successfully dropped.


In [15]:
snow.upload_dataframe(Akmishra_TECFIDERA, 'Akmishra_TECFIDERA_Enddate', schema = 'BIOGEN')

Upload Successful!


In [16]:
%%read_sql Akmishra_TECFIDERA_updated
    SELECT Distinct A.* , 
           max(B.end_date) over (partition by A.patient_suffix,A.service_date order by A.service_date) as Final_end_date
      FROM {db}.{schema}.Akmishra_TECFIDERA_Enddate    A
           inner join {db}.{schema}.Akmishra_TECFIDERA_Enddate    B
                   on A.PATIENT_SUFFIX = B.PATIENT_SUFFIX
                  and B.Service_date  <= A.service_date
     order by patient_suffix, service_date

Query started at 10:52:14 AM Eastern Daylight Time; Query executed in 0.25 m

,source,claim_number,patient_suffix,service_date,drug_code,days_supply,refills,quantity_dispensed,days_supply_final,end_date,final_end_date
0,PHARMACY,120885000000732130,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2014-11-13,64406000602,30.0,0.0,0.0,30.0,2014-12-13,2014-12-13
1,PHARMACY,118797000005181837,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2014-12-11,64406000602,30.0,0.0,0.0,30.0,2015-01-10,2015-01-10
2,PHARMACY,119712000003522834,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-01-12,64406000602,30.0,0.0,0.0,30.0,2015-02-11,2015-02-11
3,PHARMACY,120514000001906631,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-02-09,64406000602,30.0,0.0,0.0,30.0,2015-03-11,2015-03-11
4,PHARMACY,121390000007842534,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-03-09,64406000602,30.0,0.0,0.0,30.0,2015-04-08,2015-04-08
5,PHARMACY,122372000002401835,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-04-13,64406000602,30.0,0.0,0.0,30.0,2015-05-13,2015-05-13
6,PHARMACY,123206000002836340,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-05-11,64406000602,30.0,0.0,0.0,30.0,2015-06-10,2015-06-10
7,PHARMACY,124349000001780604,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-06-10,64406000602,30.0,0.0,0.0,30.0,2015-07-10,2015-07-10
8,PHARMACY,125174000004389536,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-07-06,64406000602,30.0,0.0,0.0,30.0,2015-08-05,2015-08-05
9,Albatross_EHR,17231300,++YFrY38k7/Cj6kngyYv5uwMTcXTiGF+VcbaGkgPEog=S2...,2014-07-30,64406000602,90.0,3.0,180.0,90.0,2014-10-28,2014-10-28


In [17]:
#Run the shift.  
Akmishra_TECFIDERA_updated['lag_end_date'] = Akmishra_TECFIDERA_updated.groupby(['patient_suffix'])['final_end_date'].shift(1)
Akmishra_TECFIDERA_updated['lag_end_date'] = pd.to_datetime(Akmishra_TECFIDERA_updated['lag_end_date'])

In [18]:
Akmishra_TECFIDERA_updated = Akmishra_TECFIDERA_updated.reset_index(drop=True)

In [19]:
#Review top few rows to ensure calculation is right
Akmishra_TECFIDERA_updated.head(100)

,source,claim_number,patient_suffix,service_date,drug_code,days_supply,refills,quantity_dispensed,days_supply_final,end_date,final_end_date,lag_end_date
0,PHARMACY,120885000000732130,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2014-11-13,64406000602,30.0,0.0,0.0,30.0,2014-12-13,2014-12-13,NaT
1,PHARMACY,118797000005181837,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2014-12-11,64406000602,30.0,0.0,0.0,30.0,2015-01-10,2015-01-10,2014-12-13
2,PHARMACY,119712000003522834,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-01-12,64406000602,30.0,0.0,0.0,30.0,2015-02-11,2015-02-11,2015-01-10
3,PHARMACY,120514000001906631,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-02-09,64406000602,30.0,0.0,0.0,30.0,2015-03-11,2015-03-11,2015-02-11
4,PHARMACY,121390000007842534,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-03-09,64406000602,30.0,0.0,0.0,30.0,2015-04-08,2015-04-08,2015-03-11
5,PHARMACY,122372000002401835,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-04-13,64406000602,30.0,0.0,0.0,30.0,2015-05-13,2015-05-13,2015-04-08
6,PHARMACY,123206000002836340,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-05-11,64406000602,30.0,0.0,0.0,30.0,2015-06-10,2015-06-10,2015-05-13
7,PHARMACY,124349000001780604,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-06-10,64406000602,30.0,0.0,0.0,30.0,2015-07-10,2015-07-10,2015-06-10
8,PHARMACY,125174000004389536,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-07-06,64406000602,30.0,0.0,0.0,30.0,2015-08-05,2015-08-05,2015-07-10
9,Albatross_EHR,17231300,++YFrY38k7/Cj6kngyYv5uwMTcXTiGF+VcbaGkgPEog=S2...,2014-07-30,64406000602,90.0,3.0,180.0,90.0,2014-10-28,2014-10-28,NaT


### Determine continuous records
Checking if the patient is coming for next refill on or before the enddate of prior refill.

In [20]:
Akmishra_TECFIDERA_updated['delta'] = Akmishra_TECFIDERA_updated.apply(lambda row:(row.service_date - row.lag_end_date).days , axis = 1).fillna(0)  
Akmishra_TECFIDERA_updated['delta'] = pd.to_numeric(Akmishra_TECFIDERA_updated['delta'])


In [21]:
Akmishra_TECFIDERA_updated['is_outside_buffer_60'] = Akmishra_TECFIDERA_updated.groupby('patient_suffix')['delta'].apply(lambda x: x > buffer_60) 
Akmishra_TECFIDERA_updated['is_outside_buffer_90'] = Akmishra_TECFIDERA_updated.groupby('patient_suffix')['delta'].apply(lambda x: x > buffer_90) 
Akmishra_TECFIDERA_updated['is_outside_buffer_120'] = Akmishra_TECFIDERA_updated.groupby('patient_suffix')['delta'].apply(lambda x: x > buffer_120) 
Akmishra_TECFIDERA_updated['total_60'] = Akmishra_TECFIDERA_updated.groupby('patient_suffix')['is_outside_buffer_60'].cumsum().astype(int) 
Akmishra_TECFIDERA_updated['total_90'] = Akmishra_TECFIDERA_updated.groupby('patient_suffix')['is_outside_buffer_90'].cumsum().astype(int) 
Akmishra_TECFIDERA_updated['total_120'] = Akmishra_TECFIDERA_updated.groupby('patient_suffix')['is_outside_buffer_120'].cumsum().astype(int) 


In [22]:
Akmishra_TECFIDERA_updated.head(100)

,source,claim_number,patient_suffix,service_date,drug_code,days_supply,refills,quantity_dispensed,days_supply_final,end_date,final_end_date,lag_end_date,delta,is_outside_buffer_60,is_outside_buffer_90,is_outside_buffer_120,total_60,total_90,total_120
0,PHARMACY,120885000000732130,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2014-11-13,64406000602,30.0,0.0,0.0,30.0,2014-12-13,2014-12-13,NaT,0.0,False,False,False,0,0,0
1,PHARMACY,118797000005181837,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2014-12-11,64406000602,30.0,0.0,0.0,30.0,2015-01-10,2015-01-10,2014-12-13,-2.0,False,False,False,0,0,0
2,PHARMACY,119712000003522834,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-01-12,64406000602,30.0,0.0,0.0,30.0,2015-02-11,2015-02-11,2015-01-10,2.0,False,False,False,0,0,0
3,PHARMACY,120514000001906631,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-02-09,64406000602,30.0,0.0,0.0,30.0,2015-03-11,2015-03-11,2015-02-11,-2.0,False,False,False,0,0,0
4,PHARMACY,121390000007842534,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-03-09,64406000602,30.0,0.0,0.0,30.0,2015-04-08,2015-04-08,2015-03-11,-2.0,False,False,False,0,0,0
5,PHARMACY,122372000002401835,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-04-13,64406000602,30.0,0.0,0.0,30.0,2015-05-13,2015-05-13,2015-04-08,5.0,False,False,False,0,0,0
6,PHARMACY,123206000002836340,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-05-11,64406000602,30.0,0.0,0.0,30.0,2015-06-10,2015-06-10,2015-05-13,-2.0,False,False,False,0,0,0
7,PHARMACY,124349000001780604,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-06-10,64406000602,30.0,0.0,0.0,30.0,2015-07-10,2015-07-10,2015-06-10,0.0,False,False,False,0,0,0
8,PHARMACY,125174000004389536,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-07-06,64406000602,30.0,0.0,0.0,30.0,2015-08-05,2015-08-05,2015-07-10,-4.0,False,False,False,0,0,0
9,Albatross_EHR,17231300,++YFrY38k7/Cj6kngyYv5uwMTcXTiGF+VcbaGkgPEog=S2...,2014-07-30,64406000602,90.0,3.0,180.0,90.0,2014-10-28,2014-10-28,NaT,0.0,False,False,False,0,0,0


In [23]:
Akmishra_TECFIDERA_updated['total_60']  =  Akmishra_TECFIDERA_updated['total_60'] + 1
Akmishra_TECFIDERA_updated['total_90']  =  Akmishra_TECFIDERA_updated['total_90'] + 1
Akmishra_TECFIDERA_updated['total_120'] =  Akmishra_TECFIDERA_updated['total_120'] + 1

In [24]:
Akmishra_TECFIDERA_updated.head(50)

,source,claim_number,patient_suffix,service_date,drug_code,days_supply,refills,quantity_dispensed,days_supply_final,end_date,final_end_date,lag_end_date,delta,is_outside_buffer_60,is_outside_buffer_90,is_outside_buffer_120,total_60,total_90,total_120
0,PHARMACY,120885000000732130,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2014-11-13,64406000602,30.0,0.0,0.0,30.0,2014-12-13,2014-12-13,NaT,0.0,False,False,False,1,1,1
1,PHARMACY,118797000005181837,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2014-12-11,64406000602,30.0,0.0,0.0,30.0,2015-01-10,2015-01-10,2014-12-13,-2.0,False,False,False,1,1,1
2,PHARMACY,119712000003522834,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-01-12,64406000602,30.0,0.0,0.0,30.0,2015-02-11,2015-02-11,2015-01-10,2.0,False,False,False,1,1,1
3,PHARMACY,120514000001906631,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-02-09,64406000602,30.0,0.0,0.0,30.0,2015-03-11,2015-03-11,2015-02-11,-2.0,False,False,False,1,1,1
4,PHARMACY,121390000007842534,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-03-09,64406000602,30.0,0.0,0.0,30.0,2015-04-08,2015-04-08,2015-03-11,-2.0,False,False,False,1,1,1
5,PHARMACY,122372000002401835,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-04-13,64406000602,30.0,0.0,0.0,30.0,2015-05-13,2015-05-13,2015-04-08,5.0,False,False,False,1,1,1
6,PHARMACY,123206000002836340,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-05-11,64406000602,30.0,0.0,0.0,30.0,2015-06-10,2015-06-10,2015-05-13,-2.0,False,False,False,1,1,1
7,PHARMACY,124349000001780604,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-06-10,64406000602,30.0,0.0,0.0,30.0,2015-07-10,2015-07-10,2015-06-10,0.0,False,False,False,1,1,1
8,PHARMACY,125174000004389536,++VVDQWrctCOQnfuzfdAdfaxsZikGe2MFiaydTBA9Cs=lA...,2015-07-06,64406000602,30.0,0.0,0.0,30.0,2015-08-05,2015-08-05,2015-07-10,-4.0,False,False,False,1,1,1
9,Albatross_EHR,17231300,++YFrY38k7/Cj6kngyYv5uwMTcXTiGF+VcbaGkgPEog=S2...,2014-07-30,64406000602,90.0,3.0,180.0,90.0,2014-10-28,2014-10-28,NaT,0.0,False,False,False,1,1,1


### Uploading final table to snowflake

In [25]:
snow.upload_dataframe(Akmishra_TECFIDERA_updated, 'Akmishra_TECFIDERA_full_detail_final', schema = 'BIOGEN')

Table already exists. Dropping it.
DROP TABLE IF EXISTS project_analytics.biogen.akmishra_tecfidera_full_detail_final;
Table successfully dropped!
Upload Successful!


## Final data prep

In [27]:
%%read_sql
desc table Akmishra_TECFIDERA_full_detail_final;

Query started at 10:56:19 AM Eastern Daylight Time; Query executed in 0.07 m

,name,type,kind,null?,default,primary key,unique key,check,expression,comment
0,SOURCE,VARCHAR(16777216),COLUMN,Y,None,N,N,None,None,None
1,CLAIM_NUMBER,VARCHAR(16777216),COLUMN,Y,None,N,N,None,None,None
2,PATIENT_SUFFIX,VARCHAR(16777216),COLUMN,Y,None,N,N,None,None,None
3,SERVICE_DATE,TIMESTAMP_NTZ(9),COLUMN,Y,None,N,N,None,None,None
4,DRUG_CODE,VARCHAR(16777216),COLUMN,Y,None,N,N,None,None,None
5,DAYS_SUPPLY,FLOAT,COLUMN,Y,None,N,N,None,None,None
6,REFILLS,FLOAT,COLUMN,Y,None,N,N,None,None,None
7,QUANTITY_DISPENSED,FLOAT,COLUMN,Y,None,N,N,None,None,None
8,DAYS_SUPPLY_FINAL,FLOAT,COLUMN,Y,None,N,N,None,None,None
9,END_DATE,TIMESTAMP_NTZ(9),COLUMN,Y,None,N,N,None,None,None


Add starter pack flag so we can see if the continuous period the patient likley started Tecfidera

In [28]:
%%read_sql
ALTER TABLE Akmishra_TECFIDERA_full_detail_final
        ADD starter_pack varchar(10)

Query started at 10:56:29 AM Eastern Daylight Time; Query executed in 0.07 m

,status
0,Statement executed successfully.


In [29]:
%%read_sql

UPDATE Akmishra_TECFIDERA_full_detail_final
   SET starter_pack = 1
 WHERE DRUG_CODE = '64406000501'

Query started at 10:56:33 AM Eastern Daylight Time; Query executed in 0.14 m

,number of rows updated,number of multi-joined rows updated
0,2518,0


## Pulling all the required fields

In [30]:
%%read_sql
    select count(distinct patient_suffix) as pat_count
          ,count(distinct claim_number)   as claim_count
          ,sum(starter_pack)              as start_pack
    from Akmishra_TECFIDERA_full_detail_final
    

Query started at 10:56:42 AM Eastern Daylight Time; Query executed in 0.14 m

,pat_count,claim_count,start_pack
0,4429,63566,2518.0


In [31]:
snow.select('SELECT * FROM Akmishra_TECFIDERA_full_detail_final').to_excel("C:\GitHub\Lymphocyte\Tec_cont_raw_final.xlsx",index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\GitHub\\Lymphocyte\\Tec_cont_raw_final.xlsx'

In [ ]:
%%read_sql   
    Select Distinct Source,
           patient_suffix,
           min(service_date) over (partition by patient_suffix)         AS TEC_START_DATE,
           max(enddate) over (partition by patient_suffix)              AS TEC_END_DATE,
           DATEDIFF(day,(min(service_date) over (partition by patient_suffix)),(max(enddate) over (partition by patient_suffix)))   AS Medication_enrollment_period,
           Count                                                        AS EPISODE,
           min(service_date) over (partition by patient_suffix,count)   AS EPI_START_DATE,
           max(service_date) over (partition by patient_suffix,count)   AS EPI_END_DATE,
           DATEDIFF(day,(min(service_date) over (partition by patient_suffix,count)),(max(enddate) over (partition by patient_suffix,count))) as EPISODE_PERIOD 
      FROM Akmishra_TECFIDERA_full_detail_final

In [33]:
!dir
!dir

 Volume in drive C has no label.
 Volume Serial Number is 32EB-0EFD

 Directory of C:\GitHub\Biogen_lymphocyte

04/09/2018  11:30 AM    <DIR>          .
04/09/2018  11:30 AM    <DIR>          ..
12/15/2017  12:17 PM               139 .gitignore
04/03/2018  04:15 PM    <DIR>          .ipynb_checkpoints
04/05/2018  09:50 AM                 0 .Rhistory
12/21/2017  11:26 PM           269,813 01_Biogen Lab Mappings_Cindi.sql
04/09/2018  09:31 AM            13,194 02_Biogen Lymphocytes_Albatross_Cindi.sql
04/09/2018  11:01 AM            11,353 03_Biogen Lymphocytes Pelican_Cindi.sql
12/14/2017  04:51 PM            47,846 ALC_Cohort_SQL_code.SQL
11/22/2017  11:09 AM            65,566 Biogen_Lymphocyte_SQL_Code.sql
12/14/2017  04:51 PM            18,658 Continuous_Enrollment_Code.SQL
01/29/2018  12:13 PM    <DIR>          data
12/14/2017  04:51 PM            12,963 DMT_Cohort_SQL_code_v1.sql
04/03/2018  04:15 PM           373,971 example_data.csv
12/14/2017  04:51 PM             6,717 Multiple